In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd

In [3]:
train_directory = '/kaggle/input/waste-classif/Data/archive (2)/TRAIN'

In [4]:
test_directory = '/kaggle/input/waste-classif/Data/archive (2)/TEST'

In [5]:
import tensorflow as tf

dataset = tf.keras.utils.image_dataset_from_directory(
    train_directory,
    labels="inferred",
    label_mode="categorical",
    image_size=(128, 128),
    batch_size=32,
    shuffle=True,
    seed=42,
)

Found 20545 files belonging to 3 classes.


In [6]:
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    test_directory,
    labels="inferred",
    label_mode="categorical",
    image_size=(128, 128),
    batch_size=32,
    shuffle=True,
    seed=42,
)

Found 2528 files belonging to 3 classes.


In [7]:
from functools import partial

DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size=3, strides=1,
                        padding="same", kernel_initializer="he_normal",
                        use_bias=False)

class ResidualUnit(tf.keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = tf.keras.activations.get(activation)
        self.main_layers = [
            DefaultConv2D(filters, strides=strides),
            tf.keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters),
            tf.keras.layers.BatchNormalization()
        ]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                tf.keras.layers.BatchNormalization()
            ]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

In [8]:
model = tf.keras.Sequential([
    DefaultConv2D(64, kernel_size=7, strides=2, input_shape=[128, 128, 3]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding="same"),
])
prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters

model.add(tf.keras.layers.GlobalAvgPool2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(3, activation="softmax"))

In [9]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001 , rho=0.9)
model.compile(loss="categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(
    dataset,
    validation_data=validation_dataset,
    epochs=15
)

Epoch 1/15
643/643 [==============================] - 74s 78ms/step - loss: 0.5649 - accuracy: 0.7736 - val_loss: 0.6163 - val_accuracy: 0.7199
Epoch 2/15
643/643 [==============================] - 46s 71ms/step - loss: 0.4228 - accuracy: 0.8300 - val_loss: 0.3594 - val_accuracy: 0.8619
Epoch 3/15
643/643 [==============================] - 48s 74ms/step - loss: 0.4015 - accuracy: 0.8422 - val_loss: 0.8406 - val_accuracy: 0.5581
Epoch 4/15
643/643 [==============================] - 47s 73ms/step - loss: 0.3633 - accuracy: 0.8553 - val_loss: 0.8954 - val_accuracy: 0.6119
Epoch 5/15
643/643 [==============================] - 47s 73ms/step - loss: 0.3564 - accuracy: 0.8592 - val_loss: 0.5156 - val_accuracy: 0.7991
Epoch 6/15
643/643 [==============================] - 47s 73ms/step - loss: 0.3393 - accuracy: 0.8680 - val_loss: 0.5562 - val_accuracy: 0.7231
Epoch 7/15
643/643 [==============================] - 47s 73ms/step - loss: 0.3267 - accuracy: 0.8700 - val_loss: 0.8872 - val_accuracy: